<a href="https://colab.research.google.com/github/gripen-ai/DP100/blob/main/VedioIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil





CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sppechtestmp3:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4412632%2F7580357%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T155216Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daadf932751ff7a08cad44c93b92c5b8304ecaf72518a9d8c1659f72c857c8b042bffcc0c8e07276481e8257e425324c2b0643159d93ec17012684f411d67f61256385a9b19c663d16ea7aec27e8e2795d8993d1a31494e88ade5a1af550901099612c0fb24ce91d1efe13ece1d628de28575613de4c769a0d91b430cd7097ef2326989ab45fd59caecf4aae2686dfe86e09ff9758bb4d9a6ad2d350f9cd3b0241e4c129bc19deffea3627a275d8729fa7d3554ca09cb27c2fd160b864f1d79f294dcd93a37abfba52420de370d0a09962e0b9333d8755cb3bf15e12870255a0f9beb60844a5bee4ac22e5331a7000158eb62cbb04f2fc06d82ca89ccbd2c0b38,tedvedio1min:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4413751%2F7582186%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240512%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240512T155216Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4239311b0e94f2f54578684c6a84a3f3d646b934cab297fd3ec1afa5e71e621e72f3e49b4c3489abcc073591cdc3a84f9000fbb8f50fc0d68a582f2a16c63736ccbc0f6cc874b1379f0e1adaeeb7a8d40613bb685364d0e788e069e0e4723160cb4b192a799c2cfbf8047ac8d8b4c2e79290a47d2a42ddd0842a61855b00e0f9c8ed24ae47b57285f9231eef9a6ba5cbb21569546bdc508f7556727de059ba335e5ece144db790afba17ed2c59a15d4729f51ef86cf2c0e10ec725be81754e2db91dd3afeedaba07b0af63d5c229dea028319e276a30cc02970a8a64b5703845a61a3d4a91bd430a625c0abc6510972112f47c5eb5ed5bf18b2e00324235d7f3'





KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1200293 bytes downloaded
Downloaded and uncompressed: sppechtestmp3
[==================================================] 2554802 bytes downloaded
Downloaded and uncompressed: tedvedio1min
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tedvedio1min/A one minute TEDx Talk for the digital age _ Woody Roseland _ TEDxMileHigh (1).mp4
/kaggle/input/sppechtestmp3/y2mate.com - A one minute TEDx Talk for the digital age  Woody Roseland  TEDxMileHigh.mp3


In [3]:
pip install moviepy

In [38]:
from moviepy.editor import VideoFileClip
import math

def split_video_into_audio_chunks(video_path, output_directory):
    video = VideoFileClip(video_path)
    duration = video.duration
    chunk_length = 20  # 1 minute in seconds

    num_chunks = math.ceil(duration / chunk_length)
    timestamp = pd.Timestamp.now().replace(hour=0, minute=0, second=0, microsecond=0)

    for i in range(num_chunks):
        start_time = i * chunk_length
        end_time = min((i + 1) * chunk_length, duration)
        chunk = video.subclip(start_time, end_time)


        chunk_audio_path = f"{output_directory}/audio_chunk_{i+1}_timestamp_{timestamp}.mp3"
        chunk.audio.write_audiofile(chunk_audio_path)

        timestamp += pd.Timedelta(seconds=20)

    video.close()

# RUN:
video_path = "/content/VedioIndexer/A one minute TEDx Talk for the digital age _ Woody Roseland _ TEDxMileHigh.mp4"
output_directory = "/content/VedioIndexer/output_dirctry"
split_video_into_audio_chunks(video_path, output_directory)


MoviePy - Writing audio in /content/VedioIndexer/output_dirctry/audio_chunk_1_timestamp_2024-05-12 00:00:00.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/VedioIndexer/output_dirctry/audio_chunk_2_timestamp_2024-05-12 00:00:20.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/VedioIndexer/output_dirctry/audio_chunk_3_timestamp_2024-05-12 00:00:40.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/VedioIndexer/output_dirctry/audio_chunk_4_timestamp_2024-05-12 00:01:00.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/VedioIndexer/output_dirctry/audio_chunk_5_timestamp_2024-05-12 00:01:20.mp3


MoviePy - Done.


In [6]:
from transformers import pipeline

#  speech recognition pipeline
pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-960h")


audio_file_path = "/content/VedioIndexer/y2mate.com - A one minute TEDx Talk for the digital age  Woody Roseland  TEDxMileHigh.mp3"


transcription = pipe(audio_file_path)

# Print the transcription
print(transcription)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

{'text': "AL WHAT AN AUDIENCE BUT IF 'M BEING HONEST I DON'T CARE WHAT YOU THINK OF MY TALK I DON'T I CARE WHAT THE INNERNET THINKS OF MY TALK AUSE THEY'RE THE ONES YOU GET IT SENE AND GET ASHARED AND I THINK THAT'S WHERE MOST PEOPLE GET AWRONG THEY'RE TALKING TO YOU HERE STEAD OF TALKING TO YOU RANDOM PERSON SCROLLING FACEBOOK THANKS FOR THE CLICK YOU SEE BACK IN TWO THOUSAND NINE WE ALL HAD THESE WEIRD LITTLE THINGS CALLED ATTENTION SPANS YE THEY'RE GONE THEY'RE GONE WE KILLED EM THEY THEY'RE DEAD I'M TRYING TO THINK OF THE LAST TIME I WATCHED IN EIGHTEEN MINUTES HEAD TALK IT'S BEEN YEARS LITERALLY YEARS SO IF YOU'RE GIVEN A TEAD TALK KEEP IT QUICK I'M DOING MINE IN UNDER A MINUTE I'M AT FORTY FOUR SECONDS RIGHT NOW THAT MEANS WE GOT TIME FOR ONE FINAL JOKE WHY OUR BALLOON'S SO EXPENSIVE INFLACTION"}


In [41]:
s = "audio_chunk_1_timestamp_2024-05-12 00:00:00.mp3"
s[-12:-4]

'00:00:00'

In [42]:
import os

# Directory containing the audio files
directory = "/content/VedioIndexer/output_dirctry"
text_chunks=[]
transcription_dict = {}
i=0

for filename in os.listdir(directory):

    if filename.endswith(".wav") or filename.endswith(".mp3"):

        file_path = os.path.join(directory, filename)

        text_chunk = pipe(file_path)

        text_chunks.append(text_chunk)

        transcription_dict[filename[-12:-4]] = text_chunk

        i+=1
        print("Processing:", file_path)
    else:

        print("Skipping:", filename)


Processing: /content/VedioIndexer/output_dirctry/audio_chunk_4_timestamp_2024-05-12 00:01:00.mp3
Processing: /content/VedioIndexer/output_dirctry/audio_chunk_2_timestamp_2024-05-12 00:00:20.mp3
Skipping: .ipynb_checkpoints
Processing: /content/VedioIndexer/output_dirctry/audio_chunk_5_timestamp_2024-05-12 00:01:20.mp3
Processing: /content/VedioIndexer/output_dirctry/audio_chunk_1_timestamp_2024-05-12 00:00:00.mp3
Processing: /content/VedioIndexer/output_dirctry/audio_chunk_3_timestamp_2024-05-12 00:00:40.mp3


In [23]:
len(transcription_dict)

5

In [24]:
len(transcription_dict.keys())

5

In [43]:
for i in transcription_dict.keys():
    print(f'{i}  : {transcription_dict[i]}')
    print("""\n\n------------------------------------------------------------------------------------""")


00:01:00  : {'text': 'R ONE FINAL JOKE WHY ARE BALLOONS SO EXPENSIVE INFLATION'}


------------------------------------------------------------------------------------
00:00:20  : {'text': "TO MY TALK AUSE THEY'RE THE ONES WHO GET AT SEEN AND GET A SHARED AND I THINK THAT'S WHERE MOST PEOPLE GET AWRONG THEY'RE TALKING TO YOU HERE STEAD OF TALKING TO YOU RANDOM PERSON SCROLLING FACEBOOK THANKS FOR THE CLICK YOU SEE BACK IN TWO THOUSAND NINE WE ALL HAVE THESE WEIRD LITTLE THINGS CALLED"}


------------------------------------------------------------------------------------
00:01:20  : {'text': ''}


------------------------------------------------------------------------------------
00:00:00  : {'text': "N WAW WHAT AN AUDIENCE BUT IF IM BE AN HONEST I DON'T CARE WHAT YOU THINK OF MY TALK I DON'T I CARE WHAT THE INNERNET THINKS"}


------------------------------------------------------------------------------------
00:00:40  : {'text': "ATTENTION SPANS YEP THEY'RE GONE THEY'RE GONE WE KIL

In [26]:
pip install langchain unstructured sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.2 M

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
#

In [27]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "demo_docs"

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [30]:
from langchain.docstore.document import Document

In [56]:
from langchain.docstore.document import Document
transformed_docs = []

# Loop through each filename
for source, content in transcription_dict.items():
    # langchain doc
    doc = Document(page_content= content['text'], metadata={ "vedio":"001", "timestamp": source,})
    transformed_docs.append(doc)



print(transformed_docs)


[Document(page_content='R ONE FINAL JOKE WHY ARE BALLOONS SO EXPENSIVE INFLATION', metadata={'vedio': '001', 'timestamp': '00:01:00'}), Document(page_content="TO MY TALK AUSE THEY'RE THE ONES WHO GET AT SEEN AND GET A SHARED AND I THINK THAT'S WHERE MOST PEOPLE GET AWRONG THEY'RE TALKING TO YOU HERE STEAD OF TALKING TO YOU RANDOM PERSON SCROLLING FACEBOOK THANKS FOR THE CLICK YOU SEE BACK IN TWO THOUSAND NINE WE ALL HAVE THESE WEIRD LITTLE THINGS CALLED", metadata={'vedio': '001', 'timestamp': '00:00:20'}), Document(page_content='', metadata={'vedio': '001', 'timestamp': '00:01:20'}), Document(page_content="N WAW WHAT AN AUDIENCE BUT IF IM BE AN HONEST I DON'T CARE WHAT YOU THINK OF MY TALK I DON'T I CARE WHAT THE INNERNET THINKS", metadata={'vedio': '001', 'timestamp': '00:00:00'}), Document(page_content="ATTENTION SPANS YEP THEY'RE GONE THEY'RE GONE WE KILLED HIM THEY'RE DEAD I'M TRYING TO THINK OF THE LAST TIME I WATCHED IN EIGHTEEN MINUTE HEAD TALK IT'S BEEN YEARS LITERALLY YEARS S

In [57]:

from langchain.vectorstores import Chroma
db = Chroma.from_documents(transformed_docs, embeddings)



In [58]:
query = "INFLATION"
search_result = db.similarity_search(query)
print(search_result[0].page_content)

R ONE FINAL JOKE WHY ARE BALLOONS SO EXPENSIVE INFLATION


In [52]:
# save to disk
db2 = Chroma.from_documents(transformed_docs, embeddings, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
docs = db3.similarity_search(query)
print(docs[0].page_content)

R ONE FINAL JOKE WHY ARE BALLOONS SO EXPENSIVE INFLATION


In [59]:
print(search_result[0].metadata)

{'source': '00:01:00', 'vedio': '001'}
